breast cancer


In [ ]:
import sys
import datetime
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
plt.style.use('ggplot')

In [ ]:
dataset = load_breast_cancer()
x = pd.DataFrame(dataset.data)
y = pd.DataFrame(dataset.target)
y.shape

(569, 1)

In [ ]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder
labelencoder_X_1 = LabelEncoder()
y = labelencoder_X_1.fit_transform(y)

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 0)

#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
# Regularization
l1_reg = keras.regularizers.l1(l=0.01)
RUN_NAME = "_3L_5_10_"
model = Sequential()
model.add(Dense(30, input_dim=30, activation='relu', name='input_layer'))
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu', name='layer_1', kernel_regularizer=l1_reg))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid', name='output_layer'))
model.compile(loss='binary_crossentropy', optimizer='adam', 
    metrics=['mae', 'acc'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (Dense)          (None, 30)                930       
_________________________________________________________________
dropout_4 (Dropout)          (None, 30)                0         
_________________________________________________________________
layer_1 (Dense)              (None, 50)                1550      
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
_________________________________________________________________
output_layer (Dense)         (None, 1)                 51        
Total params: 2,531
Trainable params: 2,531
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Create a TensorBoard logger
csv_logger = keras.callbacks.CSVLogger(
    'training_logs.csv', separator=',', append=False)

checkpointer = keras.callbacks.ModelCheckpoint(filepath='models/sales.model.best.model.hdf5', 
                               verbose=1, save_best_only=True, save_weights_only=False)
# filepath='/models/mnist.model.best.weights.{epoch:02d}-{val_loss:.2f}.hdf5

log_dir = "logs/fit/" + RUN_NAME +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Create EarlyStopping callback
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0.001, patience=5)

In [ ]:
hist = model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    batch_size = 10,
    epochs=150,
    shuffle=True,
    verbose=2,
    callbacks=[csv_logger, tensorboard_callback , checkpointer, early_stopping]
)

Epoch 1/150

Epoch 00001: val_loss did not improve from 0.08874
41/41 - 0s - loss: 2.3786 - mae: 0.3692 - acc: 0.7433 - val_loss: 2.0262 - val_mae: 0.2683 - val_acc: 0.9029
Epoch 2/150

Epoch 00002: val_loss did not improve from 0.08874
41/41 - 0s - loss: 1.8166 - mae: 0.2235 - acc: 0.9218 - val_loss: 1.5640 - val_mae: 0.1657 - val_acc: 0.9515
Epoch 3/150

Epoch 00003: val_loss did not improve from 0.08874
41/41 - 0s - loss: 1.4351 - mae: 0.1584 - acc: 0.9242 - val_loss: 1.2129 - val_mae: 0.1229 - val_acc: 0.9709
Epoch 4/150

Epoch 00004: val_loss did not improve from 0.08874
41/41 - 0s - loss: 1.1059 - mae: 0.1292 - acc: 0.9511 - val_loss: 0.9276 - val_mae: 0.1053 - val_acc: 0.9903
Epoch 5/150

Epoch 00005: val_loss did not improve from 0.08874
41/41 - 0s - loss: 0.8428 - mae: 0.1119 - acc: 0.9584 - val_loss: 0.6980 - val_mae: 0.0940 - val_acc: 0.9903
Epoch 6/150

Epoch 00006: val_loss did not improve from 0.08874
41/41 - 0s - loss: 0.6556 - mae: 0.1090 - acc: 0.9633 - val_loss: 0.524

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict_classes(X_test)
cm = confusion_matrix(y_test, y_pred)
print('confusion matrix is :',cm )

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
confusion matrix is : [[22  0]
 [ 0 35]]
